In [ ]:
#!/usr/bin/env python3
import numpy as np

# Predefined weight (bias) vectors and weight (coupling) matrices for basic gates.
# In these examples, the gate definitions are chosen so that the energy E = - sum_i h_i m_i - 
# sum_{i<j} J_{ij} m_i m_j is minimized for valid logic states.
#

# Buffer
Buffer_h = np.array([0,0])
Buffer_J = np.array([
    [0,1],
    [1,0]
])

# For an invertible AND gate with nodes [input1, input2, output]:
AND_h = np.array([1, 1, -2])
AND_J = np.array([
    [ 0, -1,  2],
    [-1,  0,  2],
    [ 2,  2,  0]
])

# For an invertible OR gate with nodes [input1, input2, output]:
OR_h = np.array([-1, -1, 2])
OR_J = np.array([
    [ 0,  1, -2],
    [ 1,  0, -2],
    [-2, -2,  0]
])

# Full Adder [C_in, B, A, S, C_out]
FA_h = np.array([0, 0, 0, 0, 0])
FA_J = np.array([
    [0.,  -1., -1.,  1.,  2.],
    [-1.,  0., -1.,  1.,  2.],
    [-1., -1.,  0.,  1.,  2.],
    [ 1.,  1.,  1.,  0., -2.],
    [ 2.,  2.,  2., -2.,  0.]
])

# 4 Bit FA
FA4_J = np.array([
    # A3  A2  A1  A0   B3  B2  B1  B0  Cot  C2  C1  C0  Cin  S3  S2  S1  S0
    [ 0,  0,  0,  0,  -1,  0,  0,  0,   2, -1,  0,  0,   0,  1,  0,  0,  0],
    [ 0,  0,  0,  0,   0, -1,  0,  0,   0,  2, -1,  0,   0,  0,  1,  0,  0],
    [ 0,  0,  0,  0,   0,  0, -1,  0,   0,  0,  2, -1,   0,  0,  0,  1,  0],
    [ 0,  0,  0,  0,   0,  0,  0, -1,   0,  0,  0,  2,  -1,  0,  0,  0,  1],
    [-1,  0,  0,  0,   0,  0,  0,  0,   2, -1,  0,  0,   0,  1,  0,  0,  0],
    [ 0, -1,  0,  0,   0,  0,  0,  0,   0,  2, -1,  0,   0,  0,  1,  0,  0],
    [ 0,  0, -1,  0,   0,  0,  0,  0,   0,  0,  2, -1,   0,  0,  0,  1,  0],
    [ 0,  0,  0, -1,   0,  0,  0,  0,   0,  0,  0,  2,  -1,  0,  0,  0,  1],
    [ 2,  0,  0,  0,   2,  0,  0,  0,   0,  2,  0,  0,   0, -2,  0,  0,  0],
    [-1,  2,  0,  0,  -1,  2,  0,  0,   2,  0,  2,  0,   0,  1, -2,  0,  0],
    [ 0, -1,  2,  0,   0, -1,  2,  0,   0,  2,  0,  2,   0,  0,  1, -2,  0],
    [ 0,  0, -1,  2,   0,  0, -1,  2,   0,  0,  2,  0,   2,  0,  0,  1, -2],
    [ 0,  0,  0, -1,   0,  0,  0, -1,   0,  0,  0,  2,   0,  0,  0,  0,  1],
    [ 1,  0,  0,  0,   1,  0,  0,  0,  -2,  1,  0,  0,   0,  0,  0,  0,  0],
    [ 0,  1,  0,  0,   0,  1,  0,  0,   0, -2,  1,  0,   0,  0,  0,  0,  0],
    [ 0,  0,  1,  0,   0,  0,  1,  0,   0,  0, -2,  1,   0,  0,  0,  0,  0],
    [ 0,  0,  0,  1,   0,  0,  0,  1,   0,  0,  0, -2,   1,  0,  0,  0,  0]
])
FA4_h = np.array([0]*17)


# Weight library: maps gate type (string) to its (h, J) definition.
weight_library = {
    "AND": {"h": AND_h, "J": AND_J},
    "OR":  {"h": OR_h,  "J": OR_J},
    "FA": {"h": FA_h, "J": FA_J},
    "4bFA": {"h": FA4_h, "J": FA4_J},
    "Buffer": {"h": Buffer_h, "J": Buffer_J}
}

def generate_circuit_weights(circuit):
    """
    Given a circuit description (with "nodes" and "gates"),
    compute the overall bias vector and weight matrix by summing
    the contributions from the individual gates.

    The circuit description is a dictionary with:
      - "nodes": a list of unique node names.
      - "gates": a list of gate dictionaries, each with:
           "type": gate type (e.g., "AND", "OR")
           "nodes": list of nodes for that gate (in the predefined order;
                    for AND and OR, assume order [input1, input2, output]).
    """
    nodes = circuit["nodes"]
    num_nodes = len(nodes)
    # Map each node name to an index.
    node_index = {node: idx for idx, node in enumerate(nodes)}
    
    # Initialize overall bias vector and weight matrix.
    h_total = np.zeros(num_nodes)
    J_total = np.zeros((num_nodes, num_nodes))
    
    # Process each gate.
    for gate in circuit["gates"]:
        gate_type = gate["type"]
        gate_nodes = gate["nodes"]
        if gate_type not in weight_library:
            raise ValueError(f"Gate type '{gate_type}' not defined in weight library.")
        gate_h = weight_library[gate_type]["h"]
        gate_J = weight_library[gate_type]["J"]
        
        if len(gate_nodes) != len(gate_h):
            raise ValueError(f"Gate of type '{gate_type}' expects {len(gate_h)} nodes, but got {len(gate_nodes)}.")
        
        # Add contributions from this gate into the overall Hamiltonian.
        for i, node in enumerate(gate_nodes):
            idx_i = node_index[node]
            h_total[idx_i] += gate_h[i]
            # For each pair of nodes in the gate, add the coupling.
            for j in range(i + 1, len(gate_nodes)):
                idx_j = node_index[gate_nodes[j]]
                J_total[idx_i, idx_j] += gate_J[i, j]
                J_total[idx_j, idx_i] += gate_J[j, i]  # Ensure symmetry.
    
    return h_total, J_total, node_index

def main():
    circuit = {
        "nodes": ["A0", "A1", "A2", "A3", "B0", "B1", "B2", "B3", "Cin0", "Cin1", "Cin2",
                  "P0", "P1", "P2", "P3", "P4", "P5", "P6", "P7", 
                  
                  "4bFA1_A3", "4bFA1_A2", "4bFA1_A1", "4bFA1_A0", "4bFA1_B3", "4bFA1_B2", "4bFA1_B1", "4bFA1_B0","4bFA1_Cout", 
                  "4bFA1_C2", "4bFA1_C1", "4bFA1_C0", "4bFA1_Cin", "4bFA1_S3", "4bFA1_S2", "4bFA1_S1", "4bFA1_S0",

                  "4bFA2_A3", "4bFA2_A2", "4bFA2_A1", "4bFA2_A0", "4bFA2_B3", "4bFA2_B2", "4bFA2_B1", "4bFA2_B0","4bFA2_Cout", 
                  "4bFA2_C2", "4bFA2_C1", "4bFA2_C0", "4bFA2_Cin", "4bFA2_S3", "4bFA2_S2", "4bFA2_S1", "4bFA2_S0", 

                  "4bFA3_A3", "4bFA3_A2", "4bFA3_A1", "4bFA3_A0", "4bFA3_B3", "4bFA3_B2", "4bFA3_B1", "4bFA3_B0","4bFA3_Cout", 
                  "4bFA3_C2", "4bFA3_C1", "4bFA3_C0", "4bFA3_Cin", "4bFA3_S3", "4bFA3_S2", "4bFA3_S1", "4bFA3_S0", "Buffer1"],
        "gates": [
            {"type": "AND", "nodes": ["A0", "B0", "P0"]},
            {"type": "AND", "nodes": ["A1", "B0", "4bFA1_A0"]},
            {"type": "AND", "nodes": ["A2", "B0", "4bFA1_A1"]},
            {"type": "AND", "nodes": ["A3", "B0", "4bFA1_A2"]},
            {"type": "Buffer", "nodes": ["Buffer1", "4bFA1_A3"]},
            
            {"type": "AND", "nodes": ["A0", "B1", "4bFA1_B0"]},
            {"type": "AND", "nodes": ["A1", "B1", "4bFA1_B1"]},
            {"type": "AND", "nodes": ["A2", "B1", "4bFA1_B2"]},
            {"type": "AND", "nodes": ["A3", "B1", "4bFA1_B3"]},
            {"type": "4bFA", "nodes": ["4bFA1_A3", "4bFA1_A2", "4bFA1_A1", "4bFA1_A0", "4bFA1_B3", "4bFA1_B2", "4bFA1_B1", 
                                       "4bFA1_B0", "4bFA1_Cout", "4bFA1_C2", "4bFA1_C1", "4bFA1_C0", "Cin0", "4bFA1_S3", 
                                       "4bFA1_S2", "4bFA1_S1", "P1"]},

            {"type": "AND", "nodes": ["A0", "B2", "4bFA2_B0"]},
            {"type": "AND", "nodes": ["A1", "B2", "4bFA2_B1"]},
            {"type": "AND", "nodes": ["A2", "B2", "4bFA2_B2"]},
            {"type": "AND", "nodes": ["A3", "B2", "4bFA2_B3"]},
            {"type": "4bFA", "nodes": ["4bFA1_Cout", "4bFA1_S3", "4bFA1_S2", "4bFA1_S1", "4bFA2_B3", "4bFA2_B2", "4bFA2_B1", 
                                       "4bFA2_B0", "4bFA2_Cout", "4bFA2_C2", "4bFA2_C1", "4bFA2_C0", "Cin1", "4bFA2_S3", 
                                       "4bFA2_S2", "4bFA2_S1", "P2"]},

            {"type": "AND", "nodes": ["A0", "B3", "4bFA3_B0"]},
            {"type": "AND", "nodes": ["A1", "B3", "4bFA3_B1"]},
            {"type": "AND", "nodes": ["A2", "B3", "4bFA3_B2"]},
            {"type": "AND", "nodes": ["A3", "B3", "4bFA3_B3"]},
            {"type": "4bFA", "nodes": ["4bFA2_Cout", "4bFA2_S3", "4bFA2_S2", "4bFA2_S1", "4bFA3_B3", "4bFA3_B2", "4bFA3_B1", 
                                       "4bFA3_B0", "P7", "4bFA3_C2", "4bFA3_C1", "4bFA3_C0", "Cin2", "P6", 
                                       "P5", "P4", "P3"]}
        ]
    }
    
    h_total, J_total, node_index = generate_circuit_weights(circuit)
    np.set_printoptions(threshold=np.inf, linewidth=200)
    print("Overall bias vector (h_total) in np.array format:")
    print("np.array(" + np.array2string(h_total, separator=", ") + ")")
    
    print("\nOverall weight matrix (J_total) in np.array format:")
    print("np.array(" + np.array2string(J_total, separator=", ") + ")")
    
    print("\nOrder of the matrix: {} x {}".format(len(circuit["nodes"]), len(circuit["nodes"])))
    print("\nNode order (mapping of node names to indices):")
    node_order = [None] * len(node_index)
    for node, idx in node_index.items():
        node_order[idx] = node
    print("[" + ", ".join('"' + s + '"' for s in node_order) + "]")


if __name__ == "__main__":
    main()


Overall bias vector (h_total) in np.array format:
np.array([ 4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,  0.,  0.,  0., -2.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., -2., -2., -2., -2., -2., -2., -2.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
 -2., -2., -2., -2.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., -2., -2., -2., -2.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])

Overall weight matrix (J_total) in np.array format:
np.array([[ 0.,  0.,  0.,  0., -1., -1., -1., -1.,  0.,  0.,  0.,  2.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  2.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
   0.,  0.,  0.,  0.,  2.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  2.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
 [ 0.,  0.,  0.,  0., -1., -1., -1., -1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  2.,  0.,  0.,  2.,  0.,  0.,  0.,  0.,  0.,  0.,